In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms.v2 as v2
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

import warnings

warnings.simplefilter("ignore")

torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
def preprocessing(img):
    img = img/255.0
    
    pre = v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    return pre(img)

def create_dataset(file_dir):
    return ImageFolder(
        root = file_dir,
        loader = torchvision.io.read_image,
        transform=preprocessing
    )

dataset_name = ["hiba", "isic2019", "ham10000", "pad_ufes_20"]

datasets = {}
for name in dataset_name:
    datasets[name] = create_dataset(f"./datasets/individual/{name}")
datasets["test"] = create_dataset(f"./datasets/test")
dataset_name.append("test")

In [3]:
dataloader = dict()
for name in dataset_name:
    dataloader[name] = DataLoader(datasets[name], batch_size=32, shuffle=False)

In [4]:
import torch.nn as nn
import copy

class JointModel(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.name = model

        model = torchvision.models.get_model(model, weights="DEFAULT")

        self.m = copy.deepcopy(model)

        self.ffnn = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.classifier = nn.Linear(512, 5)

    def forward(self, x):
        x = self.m(x)
    
        ret = self.ffnn(x)
        ret = self.classifier(ret)

        return ret

models = [
    JointModel(model="shufflenet_v2_x2_0"),
    JointModel(model="convnext_tiny"),
    JointModel(model="efficientnet_v2_s"),
]

for i in range(len(models)):
    models[i] = models[i].to(device)

In [5]:
for i in range(len(models)):
    models[i].load_state_dict(torch.load(f"./save_models/{models[i].name}.pt"))

for i in range(len(models)):
    for param in models[i].parameters():
        param.requires_grad = False
    models[i].eval()

In [6]:
class EnsembledModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        
        self.models = copy.deepcopy(models)
        
        for i in range(len(self.models)):
            self.models[i].classifier = nn.Identity()

        self.ffnn = nn.Sequential(
             nn.Linear(512 + 512 + 512, 2048),
             nn.ReLU(),
             nn.Dropout(),
             nn.Linear(2048, 1024),
             nn.ReLU(),             
             nn.Dropout(),
             nn.Linear(1024, 1024),
             nn.ReLU(),
             nn.Dropout(),
             nn.Linear(1024, 1024),
             nn.ReLU(),
             nn.Dropout(),
             nn.Linear(1024, 1024),
             nn.ReLU(),
             nn.Dropout(),
             nn.Linear(1024, 5),
         )

    def forward(self, x):
        x1 = self.models[0](x)
        x2 = self.models[1](x)
        x3 = self.models[2](x)
        x = torch.cat([x1, x2, x3], dim=1)
        x = self.ffnn(x)
        return x
    
model = EnsembledModel([models[i] for i in range(len(models))])
model = model.to(device)
model

EnsembledModel(
  (ffnn): Sequential(
    (0): Linear(in_features=1536, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=1024, out_features=1024, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=1024, out_features=5, bias=True)
  )
)

In [7]:
model.load_state_dict(torch.load("./save_models/ensemble/ensembled.pt"))

<All keys matched successfully>

In [8]:
from sklearn import metrics
from imblearn import metrics as immetrics
from sklearn.metrics import confusion_matrix, classification_report


disease_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "melanoma",
    "squamous cell carcinoma",
    "others"
]


def prediction(model, testloader, device):
    model.eval()
    predictions = []
    predictions_prob = []
    actual = []
    with torch.no_grad():
        for images, labels in tqdm(testloader):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            output = torch.nn.functional.softmax(output, dim=1)
            predictions.append(output.max(dim=1)[1])
            actual.append(labels)
            predictions_prob.append(output)

    actual = torch.concat(actual).cpu()
    predictions = torch.concat(predictions).cpu()
    predictions_prob = torch.concat(predictions_prob).cpu()

    return predictions, predictions_prob, actual
    
def display_results(name, model, loader, device):
    predictions, predictions_prob, actual = prediction(model, loader, device)

    print(name)
    ret = [
        metrics.accuracy_score(actual, predictions),
        metrics.precision_score(actual, predictions, average='macro'),
        metrics.recall_score(actual, predictions, average='macro'),
        metrics.f1_score(actual, predictions, average='macro'),
        immetrics.specificity_score(actual, predictions, average="macro"),
        metrics.roc_auc_score(actual, predictions_prob, average="macro", multi_class="ovr")
    ]
    
    return ['%.3f' % (n*100) for n in ret]

    
print(display_results("shufflenet_v2_x2_0", models[0], dataloader["test"], device))
print(display_results("convnext_small", models[1], dataloader["test"], device))
print(display_results("efficientnet_v2_s", models[2], dataloader["test"], device))
print(display_results("Ensemble Model", model, dataloader["test"], device))
dataset_name = ["hiba", "isic2019", "ham10000", "pad_ufes_20"]

for name in dataset_name:
    print(display_results(f"Ensemble Model,{name} Dataset", model, dataloader[name], device))

100%|██████████| 97/97 [00:09<00:00,  9.94it/s]


shufflenet_v2_x2_0
['83.814', '76.078', '73.626', '74.384', '94.256', '95.518']


100%|██████████| 97/97 [00:10<00:00,  9.33it/s]


convnext_small
['84.720', '79.630', '73.371', '76.204', '94.207', '96.134']


100%|██████████| 97/97 [00:11<00:00,  8.67it/s]


...

Ensemble Model,isic2019 Dataset
['89.126', '86.727', '80.892', '83.497', '95.742', '96.530']


100%|██████████| 6/6 [00:01<00:00,  4.28it/s]


Ensemble Model,ham10000 Dataset
['91.813', '92.048', '75.230', '81.164', '94.206', '95.992']


100%|██████████| 8/8 [00:01<00:00,  4.34it/s]

Ensemble Model,pad_ufes_20 Dataset
['85.022', '80.282', '71.868', '74.463', '95.583', '92.742']
